In [1]:
import re
import pandas as pd
from typing import List

# ========= 設定 =========
INPUT_CSV = "コマンド推論用_1000.csv"
OUTPUT_CSV = "コマンド推論用_1000_chunked.csv"
TEXT_COL = "zenbun"
CHUNK_SIZE = 1000
CARRY_COLS = ["id", "title_1"]  # 引き継ぎたい列
# =======================

def normalize_zenbun(text: str) -> str:
    """
    zenbun の前処理
    - None / NaN 対応
    - 改行・タブ・空白をすべて削除
    """
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)
    text = re.sub(r"\s+", "", text)
    return text

def build_char_chunks(
    df: pd.DataFrame,
    text_col: str,
    chunk_size: int,
    carry_cols: List[str],
) -> pd.DataFrame:
    records = []

    for doc_idx, row in df.iterrows():
        norm = normalize_zenbun(row[text_col])
        if not norm:
            continue

        for chunk_id, start in enumerate(range(0, len(norm), chunk_size)):
            rec = {
                "doc_idx": doc_idx,
                "chunk_id": chunk_id,
                "chunk_text": norm[start:start + chunk_size],
            }
            # 元の列を引き継ぐ
            for c in carry_cols:
                if c in df.columns:
                    rec[c] = row[c]

            records.append(rec)

    return pd.DataFrame(records)

# ========= 実行 =========
df = pd.read_csv(INPUT_CSV)
df_chunks = build_char_chunks(
    df,
    text_col=TEXT_COL,
    chunk_size=CHUNK_SIZE,
    carry_cols=CARRY_COLS,
)

print("元データ行数:", len(df))
print("チャンク数:", len(df_chunks))
df_chunks.head()

# CSV 出力
df_chunks.to_csv(OUTPUT_CSV, index=False)
print(f"保存しました: {OUTPUT_CSV}")


元データ行数: 1000
チャンク数: 12239
保存しました: コマンド推論用_1000_chunked.csv


In [2]:
df_chunks.head()

,doc_idx,chunk_id,chunk_text,id
0,0,0,１．調査目的と調査概要スライド番号３スライド番号４４．スケジュール（案）（参考）ｗｇコメント...,26317
1,0,1,た漏水は確認されなかった①②③漏水は確認されなかったベント管スリーブ端部サンドクッションドレ...,26317
2,1,0,原子力発電所の新規制基準適合性に係る審査会合（第１回）１．日時：平成２５年７月１６日（火）９...,34305
3,2,0,原子力発電所の新規制基準適合性に係る審査会合（第２回）１．日時：平成２５年７月２３日（火）１...,34313
4,3,0,原子力発電所の新規制基準適合性に係る審査会合（第３回）１．日時：平成２５年７月２５日（木）１...,34321
